<p style="text-align: center;font-size: 40pt">Point cloud processing</p>

Hidden custom latex commands here $ \curvearrowright$

----
[comment]: <> (General commands)
$\DeclareMathOperator*{\argmin}{arg\,min}$
$\DeclareMathOperator{\error}{error}$
$\DeclareMathOperator*{\match}{match}$
$\DeclareMathOperator{\distance}{d}$
$\DeclareMathOperator{\outlier}{outlier}$
$\DeclareMathOperator{\weight}{w}$
$\DeclareMathOperator{\datafilter}{datafilter}$
$\newcommand{\mat}[1]{\mathbf{#1}}$
$\newcommand{\point}[2][]{{}^{#1}\mathbf{#2}}$
$\newcommand{\frame}[1]{\mathcal{#1}}$
$\newcommand{\shape}[2][]{{}^{#1}\mathcal{#2}}$
$\newcommand{\matches}[1]{\mathcal{#1}}$
$\newcommand{\transformation}[3][T]{{}_{#2}^{#3}\mat{#1}}$
$\newcommand{\weights}[1]{\mathcal{#1}}$
----

# Introduction
The different types of <tt>data filters</tt> try to augment the distinctiveness of the inputs usually by reducing the number of features and by augmenting the dimension of either the features or the descriptors.
For example, a black and white image has a uniform distribution of features (a grid) and one dimension descriptor (the intensity) associated to each feature. 
After some <tt>data filters</tt> are applied, only few points in the image will be kept as features and the descriptors will be increased with information from neighboring pixels, for example to 64 dimensions when using Scale Invariant Feature Transform (SIFT) descriptors [[Lowe, 2004]](https://link.springer.com/article/10.1023%2FB%3AVISI.0000029664.99615.94).
In the case of a point cloud, it might be necessary to extract surface normal vectors (*feature enhancement*), while uniformizing the density of points (*features reduction*).
This can also be viewed as *lossy data compression*.

# Feature Enhancement
When only geometric information is available, there are still ways to extract some level of distinctness by using differential geometry.
We shortly introduce key concepts related to the use of geometric information.
In this work, the notion of a shape $\shape{S}$ is used as a representation of a generic object in the Euclidean space with a certain set of properties.
For example, those properties can be photometric, thermic, and semantic. 
Simple shapes, such as points, lines, quadrics, can be easily parametrized but most of the shapes encountered in the a real environment are too complex to be completely synthesized with parameters.
To allow a certain representation of the world, a complex shape $\shape{S}$ can be approximated by a set of other shapes only if they can be expressed in the same frame of reference $\frame{F}$.

Sensors measuring depth produce such approximations by discretizing the environment in a set of points. 
From smooth areas defined by those points, we present five types of primitives that can be extracted based on Differential Geometry: point, line, plane, curve and quadric.
The first derivative group rely on normal vectors $n$ (i.e. orthogonal to a line or plane) and tangent vectors $t$ (i.e. parallel to a line or plane) to express the area.
Since they are defined with respect to a point $\point{p}$, normal and tangent vectors can be represented with a minimal set of 2 angles in polar coordinates.
Normal and tangent vectors can be seen as a dual representation.
The choice of using either one or the other is defined by the minimum information required to express a primitive.
In the case of a line in 3D, the normal vectors needs to define a plane perpendicular to that line. 
Therefore, only using the tangent vector is more convenient.
The same reasoning holds for using a normal vector for the surface.
The notion of direction also needs to be defined depending of the primitive.
It is reasonable to use unsigned direction in the case of tangents and signed direction in the case of normals where positive sign defined the outer surface of the shape.
The motivation behind this choice is that we want to keep track of which side of a surface we are measuring while moving.
In the 2D case, it is equivalent to represent a line by a tangent or a normal in term of the number of parameters.
However, it is usually assumed that the perceived 2D plane cuts perpendicular surfaces, so it makes more sense to use normal vector to also track the outer side of the line.
[Figure 2.2](#2d_lines) illustrates this choice of representation for the 2D case.
Viewpoints $v_n$ (i.e., where the sensor was when a point $\point{p}$ was measured on a surface $\shape{S}$) are used to determine the direction of the normal vectors $n$, whereas no extra information is needed for a tangent vector $t$ laying on a line that can be observed from both sides.
<p id="2d_lines" style="text-align: center;">
    <img src="images/normals.png" width="36.75%"/>
    <img src="images/tangents.png" width="36.75%"/> <br/>
    <b>Figure 2.2:</b> Difference between using normal vectors and tangent vectors to represent a 2D shape.
    <em>Left</em>: The 2D line is known to be measured from a volume, which can only view from one side as illustrated with the black arrows $v_1$ and $v_2$.
    The direction of the surface make sense to be encoded in its representation leading to the choice of normal vectors $n$ (dashed blue arrow).
    <em>Right</em>: The same line can be observe from both direction leading to the selection of an undirected tangent vector $t$ (dashed blue line).
</p>

As for the second derivative group, a curve is parametrized by a curvature $\kappa$ (a scalar) representing an osculating circle parallel to the tangent of a line. 
In the case of a quadric (i.e. a curved surface), it is parametrized by principal direction vectors $t_{min}$, $t_{max}$ and the principal curvature scalars $\kappa_{min}, \kappa_{max}$.
Those principal directions rely on a point $\point{p}$ and on the normal vector $n$, so they could be expressed only with one angle.
In other words, the principal directions are tangent vectors to the surface complementing the normal vector.
In the case of curves, the curvature is always positive as opposed to quadrics for which a positive $\kappa$ means that the surface bend in the same direction of the normal vector and vice-versa.
For simplicity in further computation, most of the publications use normalized 3D vectors to express normals and tangents leading to a larger set of parameters, as shown in [Table 2.1](#primitivesParameters).
<p id="primitivesParameters" style="text-align: center;">
    <b>Table 2.1:</b> Comparison of different parametrizations used to represent geometric primitives. <br/>
    Minimum parametrization
</p>

| *Name* | &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; *Parameters* &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; | &nbsp;&nbsp;&nbsp; *Constraints* &nbsp;&nbsp;&nbsp; | *Reference* |
|:--------------------:|:--------------------------------------------------------:|:-------------------------------------------------------:|:------------------:|
|         Point        |                 $\point{p} = \{x, y, z\}$                |               $\point{p} \in \mathbb{R}^3$              |     $\frame{F}$    |
|        Tangent       |                  $t = \{\theta, \phi \}$                 |                $\theta \in [ -\pi, \pi )$               |     $\frame{F}$    |
|                      |                                                          |      $\phi \in [ -\frac{\pi}{2}, \frac{\pi}{2} ]$       |                    |
|        Normal        |                  $n = \{\theta, \phi \}$                 |                $\theta \in [ -\pi, \pi )$               |     $\frame{F}$    |
|                      |                                                          |      $\phi \in [ -\frac{\pi}{2}, \frac{\pi}{2} ]$       |                    |
| Principal Directions |                          $\psi$                          |                 $\psi \in [ -\pi, \pi )$                | $\{n, \frame{F}\}$ |
|       Curvature      |                         $\kappa$                         |                 $\kappa \in \mathbb{R}_+$               |     $\frame{F}$    |
| Principal Curvatures | $\kappa = \{\kappa_\mathrm{min}, \kappa_\mathrm{max} \}$ | $\kappa \in \mathbb{R}^2$ | $\{n, \frame{F}\}$ |

<p style="text-align: center;">Typical parametrization</p>

|                      |                                                         |                                 |                    |
|:--------------------:|:-------------------------------------------------------:|:-------------------------------:|:------------------:|
|         Point        |                $\point{p} = \{x, y, z\}$                |   $\point{p} \in \mathbb{R}^3$  |     $\frame{F}$    |
|        Tangent       |                 $t = \{t_x, t_y, t_z\}$                 |            $|t| = 1$            |     $\frame{F}$    |
|        Normal        |                 $n = \{n_x, n_y, n_z\}$                 |            $|n| = 1$            |     $\frame{F}$    |
| Principal Directions |      $\gamma = \{t_\mathrm{min}, t_\mathrm{max}\}$      | $n \perp t_{min} \perp t_{max}$ |     $\frame{F}$    |
|       Curvature      |                         $\kappa$                        |    $\kappa \in \mathbb{R}_+$    |     $\frame{F}$    |
| Principal Curvatures | $\kappa = \{\kappa_\mathrm{min}, \kappa_\mathrm{max}\}$ |    $\kappa \in \mathbb{R}^2$    | $\{n, \frame{F}\}$ |

Those parameters (point, tangent, normal, principal direction, curvature, and principal curvatures) bring us to a group of parametrized primitives (point, line, plane, curve and quadric), which can be helpful to approximate other complex 3D shapes. 
Those geometric primitives are listed in [Table 2.2](#primitivesDef) with their characteristics.
A graphical representation of those primitive is also showed in [Figure 2.3](#approximation1D) in the case of a shape considered as a 1D manifold and in [Figure 2.4](#approximation2D) in the case of a 2D manifold.
In their current configuration, lines, planes, curves and quadrics are unbounded, which means that they can reach infinity on their unconstrained direction. 
One can constrain a primitive by using a primitive with a lower dimensionality [[Besl, 1988]](https://ieeexplore.ieee.org/document/5966). 
Thus, a plane can be bounded by a set of lines, a line can be bounded by a set of points, etc. 
<p id="primitivesDef" style="text-align: center;margin-left: 25%;margin-right: 25%;">
    <b>Table 2.2:</b> Characteristics of primitives used for shape approximations.
    The number in parenthesis of the column <em>Nb Param.</em> corresponds to the minimum number of parameters that can be used to express the same primitive.
</p>

| *Primitives* | &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; *Parameters* &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; | *Derivative* | *Manifold* | *Bound* | *Nb Param.* |
|:------------:|:------------------------------------------:|:------------:|:----------:|:-----------:|:-----------:|
|     Point    |                 $\point{p}$                |       0      |      0     |      -      |     3(3)    |
|     Line     |          $\mathcal{L} = \{ p, t\}$         |       1      |      1     |    point    |     6(5)    |
|     Plane    |             $\Phi = \{ p, n \}$            |       1      |      2     | line, curve |     6(5)    |
|     Curve    |    $\mathcal{C} = \{ p, t, n, \kappa \}$   |       2      |      1     |    point    |    10(7)    |
|    Quadric   | $\mathcal{Q} = \{ p, n, \gamma, \kappa \}$ |       2      |      2     | line, curve |    14(8)    |

<p id="approximation1D" style="text-align: center;">
    <img src="images/primitive1D.jpg" width="50%"/> <br/>
    <b>Figure 2.3:</b> Representations of a complex shape approximated as a 1D manifold.
    Representations (in dark blue) of a complex shape $\shape{S}$ (in light gray) approximated as a 1D manifold.
    <em>Left</em>: no derivative (point).
    <em>Middle</em>: first derivative (line).
    <em>Right</em>: second derivative (curve).
</p>

<p id="approximation2D" style="text-align: center;">
    <img src="images/primitive2D.jpg" width="50%"/> <br/>
    <b>Figure 2.4:</b> Representations of a complex shape approximated as a 1D manifold.
    Representations (in dark blue) of a complex shape $\shape{S}$ (in light gray) approximated as a 2D manifold.
    <em>Left</em>: no derivative (point).
    <em>Middle</em>: first derivative (plane).
    <em>Right</em>: second derivative (quadric).
</p>

At a higher level of organization, a group of geometric primitives can be processed without any proximity assumption (*unstructured*) or with some smoothness constrains (*structured*).
Examples of representation for a 1D manifold is a spline, while for a 2D manifold a mesh or Non-uniform rational B-spline (NURBS) can be used.
When it comes to a noisy group of points, tensor voting [[Medioni et al., 2000]](http://www.sci.utah.edu/~gerig/CS7960-S2010/handouts/Medioni_tensor_voting.pdf) can be used to interpolate shape on a dense 3D grid.
The voting results can then be later process to extract 1D and 2D manifolds out of the dense volume.

Higher derivatives could also approximate a shape at one point with more precision. 
Unfortunately, high derivatives are very sensitive to noise when applied outside of a theoretical context.
In this review, we limit ourselves to the second derivative given that a non-negligible noise level is expected from the sensor measurements and from the motion of the sensor.
As shown in [[Pomerleau et al., 2012a]](https://ieeexplore.ieee.org/document/6473358), even the first derivative needs a large supporting surface to overcome the sensor noise of typical laser rangefinders.
For example, extracting the surface normal from a flat area of 10-cm radius already leads to an expected error of 1.6$^\circ$ (0.03 rad) for the rangefinder Sick LMS-151.
This is one of the characteristics of robotic applications when it comes to selecting a surface parametrization.
The ratio of noise to signal is often higher in robotics than in object modeling, thus rendering many complex modeling algorithms ineffective.
This could explain why most registration algorithms applied to robotics tend to select a shape representation very close to the raw measurements (i.e., points) instead of relying on faulty surface reconstruction.

## Sensitivity to transformation functions
The shape representations are affected differently by transformation functions.
At a more generic level, transformation functions affect geometric quantities.
Examples of quantities are: coordinate, orientation, length, angle, and length ratio.
Those geometric quantities with examples of associated primitives are listed in [Table 2.3](#transformationQuanties).
As examples of lengths, we used $\kappa$, which is the inverse of a radius, and the eigenvalues $\lambda$, which define a scale over a vector.
Having geometric parameters invariant to as many transformations as possible helps the matching function during registration because the association will be less sensitive to large initial alignment error.
[Table 2.4](#influenceFunc) relates the different geometric quantities to the basic transformation functions affecting them.
<p id="transformationQuanties" style="text-align: center;margin-left: 25%;margin-right: 25%;">
    <b>Table 2.3:</b> Examples of geometric quantities susceptible to be affected by a transformation function.
</p>

|  *Quantity*  |    &nbsp; *Single Entity* &nbsp;    |     *Relationship in a Set*    |
|:------------:|:-----------------------------------:|:------------------------------:|
|  Coordinate  |             $\point{p}$             |                -               |
|  Orientation | $n, t_\mathrm{min}, t_\mathrm{max}$ |                -               |
|    Length    |          $\kappa, \lambda$          |         $||p_a - p_b||$        |
|     Angle    |                  -                  | $\text{arccos}(n_a \cdot n_b)$ |
| Length Ratio |                  -                  |      $\kappa_a / \kappa_b$     |

<p id="influenceFunc" style="text-align: center;margin-left: 25%;margin-right: 25%;">
    <b>Table 2.4:</b> Influence of a transformation function on quantities defining a geometric primitive.
    Influence of a transformation function on quantities defining a geometric primitive.
    Cells marked with a "X" mean that the transformation affects the values of the entity.
</p>

|       *Function*       | *Coordinate* | *Length* | *Orientation* | *Angle* | *Length Ratio* |
|:----------------------:|:------------:|:--------:|:-------------:|:-------:|:--------------:|
|       Translation      |       X      |     -    |       -       |    -    |        -       |
|     Uniform Scaling    |       X      |     X    |       -       |    -    |        -       |
|        Rotation        |       X      |     -    |       X       |    -    |        -       |
|   Nonuniform Scaling   |       X      |     X    |       X       |    X    |        -       |
|          Shear         |       X      |     X    |       X       |    X    |        X       |
|  Orthogonal Projection |       X      |     X    |       X       |    X    |        X       |
| Perspective Projection |       X      |     X    |       X       |    X    |        X       |

Most of the time, point cloud features come without external descriptors (i.e. as opposed to an image containing photometric information), so the proximity of other features is used to extend the shape approximation to support further derivatives. 
Surface orientations (or line orientations in 2D) are mainly used in literature [\[Pulli, 1999](https://ieeexplore.ieee.org/document/805346), [Censi,
2008](https://ieeexplore.ieee.org/document/4543181), [Bosse and Zlot, 2009b](https://ieeexplore.ieee.org/document/5152851), [Jost and Hugli, 2002](https://ieeexplore.ieee.org/document/1024114), [Schutz et al., 1998](https://ieeexplore.ieee.org/document/711852),
[Jost and Hügli, 2002](https://link.springer.com/chapter/10.1007/3-540-45783-6_12), [Segal et al., 2009\]](http://www.roboticsproceedings.org/rss05/p21.html). 
Line orientations are also used in image registration where the environment presents very few salient points when considering only intensity variation [[Stewart et al.,
2003]](https://ieeexplore.ieee.org/document/1242341). 
Work based on surface normal vector distributions of surrounding points are also used by [Magnusson et al. [2009]](https://ieeexplore.ieee.org/document/5152538) and [Fairfield and Wettergreen [2009]](https://ieeexplore.ieee.org/document/5152688).

# Descriptor Enhancement
A comparison of descriptors extracted from 2D point clouds can be found in [[Bosse and Zlot, 2009a]](https://www.sciencedirect.com/science/article/abs/pii/S0921889009000992).
It is proposed that moment grid is better than 2D shape context, Gestalt, Hough transform peaks, orientation and projection histograms, and normal orientation histogram grid. 
Extension to the 2D shape context can be found in [[Tsai et al., 2010]](https://ieeexplore.ieee.org/document/5223602). 
Another study for 3D point clouds also concludes that moment grid is better than 3D shape context, spin image, shell image and local covariance [[Bosse and Zlot, 2009b]](https://ieeexplore.ieee.org/document/5152851).

Usually, Iterative Closest Point (ICP) is done using only geometric features, but some works also present results using the intensity remission from an Hokuyo [[Yoshitaka et al., 2006]](https://ieeexplore.ieee.org/document/4153430) and from specialized system using three different wavelengths [[Godin et al., 1994]](https://www.spiedigitallibrary.org/conference-proceedings-of-spie/2350/1/Three-dimensional-registration-using-range-and-intensity-information/10.1117/12.189139.short). 
Laser range finders are also combined with cameras to add color information to measured points [\[Schutz et al., 1998](https://ieeexplore.ieee.org/document/711852), [Druon et al., 2006\]](https://ieeexplore.ieee.org/document/4097937).
When other sensors are used to provide descriptors, calibration is required.
Interestingly, calibration also relies on registration solutions.
Terrestrial survey scanners often have a calibrated camera associating color to 3D points, similar to RGB-D cameras.
With the larger availability of photometric information, descriptors developed by the computer vision community can be used.
In [[Tuytelaars and Mikolajczyk, 2008]](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.83.7050&rep=rep1&type=pdf), characteristics of descriptive features are listed as *rotation*, *scale*, and *affine invariance*.
Evaluation criteria are listed as *repeatability*, *distinctiveness*, *locality*, *quantity*, *accuracy*, and *efficiency*. 
In image registration, the list of most common tools for extracting descriptors are: Harris, Hessian, SUSAN, Harris-Laplace, Hessian-Laplace, DoG (SIFT), SURF, Harris-Affine, Hessian-Affine, Salient Regions, Edge-based, MSER, Intensity-based and Superpixel [[Tuytelaars and Mikolajczyk, 2008]](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.83.7050&rep=rep1&type=pdf).
It is interesting to note that descriptors based on photometric information rely on passive illumination to ensure invariance.
This assumption of illumination sources remaining static, which is mostly true for indoor lights, needs to be treated carefully for outdoor illumination, where the sun moves and clouds can shade light.
Laser's intensity measurements are even more sensitive to transformation functions because the illumination source follows the sensor position.

# Feature Reduction
In applications using point clouds, features are sparse but not uniformly distributed. 
Nevertheless, the fact that sensors can provide a huge number of readings on a short period of time creates a bottleneck in term of computation power for the association as explained later.
Several techniques are used to reduce the number of features: random sampling [\[Jost and Hugli, 2002](https://ieeexplore.ieee.org/document/1024114), [Pan et al., 2010\]](https://ieeexplore.ieee.org/document/5476132), uniform grid [\[Magnusson et al., 2009](https://ieeexplore.ieee.org/document/5152538), [Bosse and Zlot, 2009b\]](https://ieeexplore.ieee.org/document/5152851), grid projection [[Pan et al., 2010]](https://ieeexplore.ieee.org/document/5476132), octree [\[Fairfield and Wettergreen, 2009](https://ieeexplore.ieee.org/document/5152688), [Wurm et al., 2010\]](http://www2.informatik.uni-freiburg.de/~stachnis/pdf/wurm10icraws.pdf), and bounding box [\[Stewart et al., 2003](https://ieeexplore.ieee.org/document/1242341), [Tsai et al., 2010\]](https://ieeexplore.ieee.org/document/5223602). 
All these techniques reduce the number of features without considering their distinctiveness. 

Having that criteria in mind, [Bosse and Zlot [2009a]](https://www.sciencedirect.com/science/article/abs/pii/S0921889009000992) present results showing that keeping a representative point per curvature cluster is better than segment centroids and mean-shift for 2D point clouds.
Also, [Gelfand et al. [2003]](https://ieeexplore.ieee.org/document/1240258) propose a sampling method selecting points leading to a better geometric stability of the minimization. 
Relying on non-geometric information, [Druon et al. [2006]](https://ieeexplore.ieee.org/document/4097937) use seven clusters based on hue values and select only one cluster carrying the most information for registration. 
The same type of solution was previously presented before by [Godin et al. [1994]](https://www.spiedigitallibrary.org/conference-proceedings-of-spie/2350/1/Three-dimensional-registration-using-range-and-intensity-information/10.1117/12.189139.short), who clustered point based on laser intensity values.

It is also possible to find more application-specific methods in the literature. 
For example, the tip of the nose, inner corners of the eyes, and nose corners are directly extracted for face detection [[Pan et al., 2010]](https://ieeexplore.ieee.org/document/5476132). 
In medical imagery, blood vessel crossings are also used to reduce features. 
Moreover, the main orientation of the blood vessel crossings and its number of branching is used to construct descriptors [[Stewart et al., 2003]](https://ieeexplore.ieee.org/document/1242341).
The complete point cloud can also be reduced to its first and second statistical moments [[Liu, 2010]](https://ieeexplore.ieee.org/document/5291420) or with orientation and projection histograms [[Bosse and Zlot, 2008]](https://journals.sagepub.com/doi/10.1177/0278364908091366).

# Sensor Noise
Sensor noise is also taken into account at this stage of the process.
Models representing noises are intended to evaluate the uncertainty of a measured point based on the limitations of the sensor used.
They may try to identify if a point is a measurement artifact or how accurately the position is measured.
To cope with stereo reconstruction noise, [Diebel et al. [2004]](https://ieeexplore.ieee.org/document/1389948) remove points with distance and surface angle to neighbors larger than two times the median of all distances and surface angles within the point cloud. 
When using laser's remission intensity, which is not invariant to distance and angle, [Yoshitaka et al. [2006]](https://ieeexplore.ieee.org/document/4153430) propose to keep points only close to the laser to reduce the impact of distance on the intensity value. 
For color images, points with low saturation value tend to be gray and are removed before applying clustering technique based on hue [[Druon et al., 2006]](https://ieeexplore.ieee.org/document/4097937). 
Points on boundaries of the sensor reading can also be removed to avoid misleading interpretation of neighbor points [[Armesto et al., 2010]](https://ieeexplore.ieee.org/document/5509371).
When an error model is available, it is also possible to add noise information based on measurement distance, incidence angle, reflectivity, etc.
Examples of noise models on distance reading are investigated for Sick LMS-151, Hokuyo URG and UTM in [[Pomerleau et al., 2012a]](https://ieeexplore.ieee.org/document/6473358).

## Example 1
The simplest of the filters is a random subsampling in order to decimate the point cloud:

$$
    \shape{P'} = \datafilter(\shape{P}) = \left\{\point{p} \in \shape{P}: \eta(\point{p})<\theta\right\}
$$

where $\eta\in[0,1)$ is a uniform-distributed random value and $\theta\in[0,1]$ a fixed threshold, corresponding to the fraction of points to keep.

## Example 2
A second example is the computation of normal vectors in a point cloud:

$$
    \shape{P'} = \datafilter(\shape{P}) = \left\{
        \left[\begin{array}{c}\point{p}\\n\end{array}\right]:
        \forall\point{p} \in \shape{P}, n=\text{normal}(\point{p})
    \right\}
$$

where $\text{normal}(\point{p})$ is the normal vector inferred around point $\point{p}$.

# Conclusion
TODO